In [1]:
from sklearn.datasets import load_digits
from torch import nn
from torch.utils.data import *
from matplotlib import pyplot as plt
import torch
from torch import Tensor, tensor
import numpy as np
import pandas as pd
import torchvision

In [2]:
mnist = load_digits()
data = mnist['data']
target = mnist['target']
data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [3]:
class Landau(Dataset):
    
    def __init__(self):
        mnist = load_digits()
        self.data = mnist['data']
        self.target = mnist['target']
        
    def __len__(self):
        return len(self.target)
    def __getitem__(self, idx):
        return tensor(self.data[idx]), tensor(self.target[idx])

In [4]:
l = Landau()

In [51]:
for b in DataLoader(l):
    print(b[0].shape)
    break

torch.Size([1, 64])


In [128]:

DATA_PATH = "C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST"
BATCH_SIZE = 256

input_size = 784
hidden_size = 240
num_classes = 10
num_epochs = 150
learning_rate = 0.001

In [31]:
train_data_loader,_ = get_data_loaders(100,100)

Extracting C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw\train-images-idx3-ubyte.gz to C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw


Extracting C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw


Extracting C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw



Extracting C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST\MNIST\raw
Processing...
Done!


In [34]:
train_data_loader

Dataset MNIST
    Number of datapoints: 784
    Root location: C:/Users/OFIR/Documents/Training/DS - Common Ground/Drive 9 - NN/Pytorch/Data/MNIST
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
from torch.utils.tensorboard import SummaryWriter

In [7]:
from tqdm.notebook import tqdm_notebook

In [8]:
import os

In [9]:
os.makedirs('./tensorboard', exist_ok=True)
writer = SummaryWriter(log_dir="./tensorboard")
ITERATION_NUM = 100


for num_iter in range(ITERATION_NUM):
    writer.add_scalar("Loss/train_loss", np.random.uniform(0,1), num_iter)
    writer.add_scalar("Loss/val_loss", np.random.uniform(0,1), num_iter)
    writer.add_scalar("Accuracy/train_loss", np.random.uniform(0,1), num_iter)
    writer.add_scalar("Accuracy/val_loss", np.random.uniform(0,1), num_iter)
    
for i, img in tqdm_notebook(enumerate(mnist.images[:100])):
    fig = plt.figure()
    plt.imshow(img)
    writer.add_figure("target = {}".format(mnist['target'][i]), fig, global_step=i)
#     break

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [10]:
from torch import optim
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [129]:
os.makedirs('./tensorboard', exist_ok=True)
writer = SummaryWriter(log_dir="./tensorboard")

model = nn.Sequential(
    nn.Linear(64,100),
    nn.ReLU(),
    nn.Linear(100,10),
    nn.ReLU(),
    nn.Softmax(),
)

torch.set_default_tensor_type(torch.FloatTensor)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

global_iter = 0 
for epoch in tqdm_notebook(range(num_epochs)):
    preds = []
    preds_proba = []
    true_labels = []
    iterator = iter(DataLoader(l, batch_size=BATCH_SIZE))
    for i, (X_train, y_train) in enumerate(iterator):
        true_labels = true_labels+list(y_train)
        y_preds = model(tensor(X_train, dtype=torch.float))
        chosen_class = y_preds.detach().numpy().argmax(axis=1)
        preds=preds+list(chosen_class)
        preds_proba = preds_proba+list(y_preds.detach().numpy()) # for PRs
        curr_loss = criterion(y_preds, tensor(y_train, dtype=torch.long)) # NOTE: preds should be first
        writer.add_scalar("Loss/train_loss", curr_loss, global_step=epoch*len(l)+i)
        
        curr_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    fig = plt.figure()
    sns.heatmap(confusion_matrix(y_pred=preds, y_true=true_labels))
    
    writer.add_figure("Confusion Matrix", fig, global_step=epoch)
    
    # pr_curve needs a binray label!
    for target in list(set([el.item() for el in true_labels])):
        rel_probas = [preds_proba[i][target] for i in range(len(preds_proba))]
        rel_real = [1 if el==target else 0 for el in [el.item() for el in true_labels]]
        writer.add_pr_curve(tag="PR Curve - Target = {}".format(target), 
                            labels=tensor(rel_real), 
                            predictions=tensor(rel_probas), 
                            global_step=epoch)

C:\Users\OFIR\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\OFIR\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [17]:
from PIL import Image
import numpy as np
import torch
import torchvision.transforms.functional as TF
import torchvision
im = Image.open("chest_xray/train/NORMAL/IM-0115-0001.jpeg")
# TF.to_tensor(im)
torchvision.transforms.ToTensor()(im).shape

torch.Size([1, 1858, 2090])